In [127]:
# install your library

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import graphviz as gr
import random

In [128]:
path = "C:/Users/zjy97/Downloads/python-causality-handbook-v1.0/matheusfacure-python-causality-handbook-f666303/causal-inference-for-the-brave-and-true/data/"

In [160]:
# 1. what is propensity score
# you dont need to directly control for confounders X to achieve conditional indepence Y1,Y0 with T|X
# it is sufficient to control for a balancing score E[T|X], and this is often the conditional probability of the treatment, P(T|X)

# then we have (Y1,Y0) independent with T|e(x)

# 2. let's see how propensity score estimation works
data = pd.read_csv(path+"learning_mindset.csv")

categ = ["ethnicity", "gender", "school_urbanicity"]
cont = ["school_mindset", "school_achievement", "school_ethnic_minority", "school_poverty", "school_size"]

data_with_categ = pd.concat([
    data.drop(columns=categ), # dataset without the categorical features
    pd.get_dummies(data[categ], columns=categ, drop_first=False)# categorical features converted to dummies
], axis=1)

print(data_with_categ.shape)

(10391, 32)


In [161]:
# estimate propensity score using logistic regression
from sklearn.linear_model import LogisticRegression

T = 'intervention'
Y = 'achievement_score'
X = data_with_categ.columns.drop(['schoolid', T, Y])

ps_model = LogisticRegression(C=1e6).fit(data_with_categ[X], data_with_categ[T])

data_ps = data.assign(propensity_score=ps_model.predict_proba(data_with_categ[X])[:, 1])

data_ps[["intervention", "achievement_score", "propensity_score"]].head()

,intervention,achievement_score,propensity_score
0,1,0.277359,0.315487
1,1,-0.449646,0.263800
2,1,0.769703,0.344024
3,1,-0.121763,0.344024
4,1,1.526147,0.367784


In [163]:
# then, weigh each propensity score by types of treatment
weight_t = 1/data_ps.query("intervention==1")["propensity_score"]
weight_nt = 1/(1-data_ps.query("intervention==0")["propensity_score"])
print("Original Sample Size", data.shape[0])
print("Treated Population Sample Size", sum(weight_t))
print("Untreated Population Sample Size", sum(weight_nt))

Original Sample Size 10391
Treated Population Sample Size 10388.57453106489
Untreated Population Sample Size 10391.439059342569


In [164]:
# another weight to interact with different treatment
weight = ((data_ps["intervention"]-data_ps["propensity_score"]) /
          (data_ps["propensity_score"]*(1-data_ps["propensity_score"])))

y1 = sum(data_ps.query("intervention==1")["achievement_score"]*weight_t) / len(data)
y0 = sum(data_ps.query("intervention==0")["achievement_score"]*weight_nt) / len(data)

ate = np.mean(weight * data_ps["achievement_score"])

print("Y1:", y1)
print("Y0:", y0)
print("ATE", ate)

# this method is called inverse probability of treatmenty weighting
# propensity score weighting is saying that we should expect treated individuals to be .38 sd above their untreated fellows

Y1: 0.2595808315593797
Y0: -0.1289233440296743
ATE 0.38850417558905403


In [165]:
# 2. standard error of ATE using propensity score with IPTW
from joblib import Parallel, delayed # for parallel processing

# define function that computes the IPTW estimator
def run_ps(df, X, T, y):
    # estimate the propensity score
    ps = LogisticRegression(C=1e6, max_iter=1000).fit(df[X], df[T]).predict_proba(df[X])[:, 1]
    
    weight = (df[T]-ps) / (ps*(1-ps)) # define the weights
    return np.mean(weight * df[y]) # compute the ATE

np.random.seed(88)
# run 1000 bootstrap samples
bootstrap_sample = 1000
ates = Parallel(n_jobs=4)(delayed(run_ps)(data_with_categ.sample(frac=1, replace=True), X, T, Y)
                          for _ in range(bootstrap_sample))
ates = np.array(ates)

In [166]:
print(f"ATE: {ates.mean()}")
print(f"95% C.I.: {(np.percentile(ates, 2.5), np.percentile(ates, 97.5))}")# ate is 0.38 and 95% CI is (0.3545132414290843, 0.41992560836402076)

ATE: 0.38774590065633185
95% C.I.: (0.3545141663026032, 0.41992559665045304)


In [ ]:
# 3. common issues with propensity score
# actually the predictive quality of the propensity score does not translate into its balancing properties
# what we want to do on propensity score is to produce it as a intermiate variable and get rid of confounders, not prediction
# even though we use fancy ml algo it can get good prediction
# propensity score doesnt have to predict the treatment very well, it just needs to include all the confounding variables
# if we incldue variables that are very good in predicting the treatment but have no bearing on the outcome it will actually increase the variance of the propensity score estimator
# this is similar to the problem linear regression faces when we include variables correlated with the treatment but not with the outcome

In [167]:
# 4. propensity score matching
# we can think of propensity score as performing dimensionality reduction on the feature space
# so previously we use inverse propensity score as weight times the outcome to get the ATE, it is good but the question is
# propensity score can be a combination of confounders, but we havd SE on IPTW, then we have to compute CI and mean.
# why do you not directly control the ps, then run with the treatment to predict outcome
cm = CausalModel(
    Y=data_ps["achievement_score"].values, 
    D=data_ps["intervention"].values, 
    X=data_ps[["propensity_score"]].values
)

cm.est_via_matching(matches=1, bias_adj=True)

print(cm.estimates)
# so right now we have ATE 0.389, it is about same with the pstw
# actually we also can drive standard deviation for ATE but bootstrap doesnt work with matching, there is no py lib for correct std

# the reason is about inconsistency(bootstrapping cannot correclt poroducing the distribution of ATE), randomness in results(if treatemnet group is small, matching results in difference and randomness is huge) and estimation erros(cannot decide for how close for unit with score) 
# some researchers say that we could use Monte Carlo simulations


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.389      0.025     15.595      0.000      0.340      0.438
           ATC      0.380      0.027     13.907      0.000      0.327      0.434
           ATT      0.406      0.027     15.226      0.000      0.354      0.459

